In [84]:
import numpy as np
import pandas as pd
from typing import Dict

In [92]:
# Import data and preview
raw_data = pd.read_pickle("data.pkl")
raw_data[0:5]

,subject,exp,phasename,num_rep,num_trial,num_formed,num_uttered,prompt,in_or_out,date_run,time_run,block,pitches
0,20,PU,5,44,1,2637,70,HECK,O,20170127,11,3,0 134.355450 1 134.360625 2 134...
1,20,PU,5,36,1,2629,62,HECK,O,20170127,11,3,0 130.217779 1 130.179978 2 130...
2,20,PU,7,20,1,2713,146,HECK,I,20170127,11,3,0 132.486939 1 132.479573 2 132...
3,20,PU,6,6,1,2649,82,HECK,O,20170127,11,3,0 139.037150 1 139.055242 2 139...
4,20,PU,7,8,1,2701,134,HECK,I,20170127,11,3,0 130.927041 1 130.913059 2 130...


In [163]:
# Generate summary statistics for each trial
agg: Dict[int, float] = {
    "count": [],
    "avg": [],
    "std": [],
    "minimum": [],
    "med": [],
    "maximum": [],
}

for pitches in raw_data.pitches:
    (
        count,
        avg,
        std,
        minimum,
        _,
        med,
        _,
        maximum,
                ) = pitches.describe()
    agg["count"].append(count)
    agg["avg"].append(avg)
    agg["std"].append(std)
    agg["minimum"].append(minimum)
    agg["med"].append(med)
    agg["maximum"].append(maximum)

In [164]:
# Preview summary statistics
summary_stats = pd.DataFrame(agg)
summary_stats[0:5]

,count,avg,std,minimum,med,maximum
0,491.0,133.610873,1.544445,124.930762,133.915850,134.887327
1,531.0,128.870786,1.549235,124.541467,129.371750,132.296456
2,461.0,129.790462,2.313823,126.706926,129.097929,135.363120
3,461.0,137.901782,0.786993,135.052405,137.730314,140.124944
4,441.0,125.673118,2.379921,121.275458,125.697445,131.048169


In [195]:
# Relativize F0
subjects = raw_data.subject.unique()

# Subset only start phase trials with the prompt "HECK"
trial_f0 = summary_stats.avg[(raw_data.phasename == 4) & (raw_data.prompt == "HECK")]
trial_subject = raw_data.subject[(raw_data.phasename == 4) & (raw_data.prompt == "HECK")] 

# Get the baseline F0 for each subject
f0 = {}
for subject in subjects:
    trials = trial_f0[subject == trial_subject]
    subject_f0 = trials.mean()
    f0[subject] = subject_f0

# Subtract pitches from baseline F0 
f0_corrected_data = []
for subject in subjects:
    

In [198]:
f0.keys()

dict_keys([20, 10, 32, 30, 5, 6, 26, 11, 33, 2, 21, 9, 7, 13, 35, 16, 17, 25, 3, 24, 12, 34, 15, 29, 31, 27, 14, 4, 19, 8, 28])

In [166]:
trial_f0[0:5]

6     122.466190
9     119.274433
14    120.328925
27    121.871277
29    122.685100
Name: avg, dtype: float64

In [133]:
sub.prompt.unique()

array(['HECK', 'HEAT', 'HIT', 'HUT', 'HOT', 'HOOT', 'HAT'], dtype=object)

In [ ]:
sub = raw_data[raw_data.phasename == 4]
#prompts = sub.prompt.unique()
sum = 0
for prompt in prompts:
    count = len(sub[sub.prompt == prompt])
    sum += count
    #print(prompt, len(sub[sub.prompt == prompt]))
print(sum)